In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the dataset
pd.set_option('display.max_columns', None)

df1 = pd.read_csv('lung_cancer_mortality_data_large_v2.csv')


# Explore the Data

In [ ]:
df1.head()

In [ ]:
df1.columns

In [ ]:
df1.shape

In [ ]:
df1.size

In [ ]:
# check for unique values
df1['treatment_type'].unique()

In [ ]:
# check for duplicates
df1.duplicated().sum()

In [ ]:
# check for missing values
df1.isna().sum()

In [ ]:
# check the data types
df1.dtypes

In [ ]:
df1.info()

In [ ]:
df1.describe()

# Data cleaning

In [13]:
# make a copy in order not to tamper with the original dataset
df = df1.copy()

In [14]:
# Convert date columns to datetime
df['diagnosis_date'] = pd.to_datetime(df['diagnosis_date'])
df['end_treatment_date'] = pd.to_datetime(df['end_treatment_date'])

In [15]:
# convert age data type to integer
df['age'] = df['age'].astype(int)

In [ ]:
# re-check data types
df.dtypes

In [ ]:
# Display dataframe
df.head()

# Feature Engineering

In [18]:
# map all columns with 1 and 0 to 'Yes' and 'No' in order to maintain consistency throughout the dataset
df['hypertension'] = df['hypertension'].map({1: 'Yes', 0: 'No'})
df['asthma'] = df['asthma'].map({1: 'Yes', 0: 'No'})
df['cirrhosis'] = df['cirrhosis'].map({1: 'Yes', 0: 'No'})
df['other_cancer'] = df['other_cancer'].map({1: 'Yes', 0: 'No'})
df['survived'] = df['survived'].map({1: 'Yes', 0: 'No'})

In [19]:
# create a new column and calculate number of days under treatment
df['days_under_treatment'] = (df['end_treatment_date'] - df['diagnosis_date']).dt.days

In [ ]:
df.head()

# Exploratory Data Analysis (EDA)

##### We want to use sqlite and because sqlite does not recognise datetime data type, we will have to convert those columns to string for this purpose

In [21]:
df['diagnosis_date'] = df['diagnosis_date'].astype(str)
df['end_treatment_date'] = df['end_treatment_date'].astype(str)

In [ ]:
df.dtypes

In [ ]:
import sqlite3

# Connect to SQLite database (or create it)
conn = sqlite3.connect('cancer_data.db')

# Insert DataFrame into SQLite database
df.to_sql('patients', conn, if_exists='replace', index=False)

# Close the connection
# conn.close()

#### Age Range Distribution of Lung Cancer Patients

In [ ]:
# Query to get age range counts for survivors and non-survivors
query_age_range = """
SELECT
    CASE
        WHEN age BETWEEN 0 AND 19 THEN '0-19'
        WHEN age BETWEEN 20 AND 29 THEN '20-29'
        WHEN age BETWEEN 30 AND 39 THEN '30-39'
        WHEN age BETWEEN 40 AND 49 THEN '40-49'
        WHEN age BETWEEN 50 AND 59 THEN '50-59'
        WHEN age BETWEEN 60 AND 69 THEN '60-69'
        WHEN age BETWEEN 70 AND 79 THEN '70-79'
        WHEN age >= 80 THEN '80+'
    END AS age_range,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 1 END) as total_count
FROM patients
GROUP BY age_range
ORDER BY age_range;
"""

# Execute the query and load the data into a DataFrame
df_age_range = pd.read_sql(query_age_range, conn)

# Display the DataFrame
df_age_range

In [ ]:
# Query to get age range counts for survivors and non-survivors
query_age_range = """
SELECT
    CASE
        WHEN age BETWEEN 0 AND 19 THEN '0-19'
        WHEN age BETWEEN 20 AND 29 THEN '20-29'
        WHEN age BETWEEN 30 AND 39 THEN '30-39'
        WHEN age BETWEEN 40 AND 49 THEN '40-49'
        WHEN age BETWEEN 50 AND 59 THEN '50-59'
        WHEN age BETWEEN 60 AND 69 THEN '60-69'
        WHEN age BETWEEN 70 AND 79 THEN '70-79'
        WHEN age >= 80 THEN '80+'
    END AS age_range,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 1 END) as total_count
FROM patients
GROUP BY age_range
ORDER BY age_range;
"""

# Execute the query and load the data into a DataFrame
df_age_range = pd.read_sql(query_age_range, conn)

# Display the DataFrame
df_age_range

In [ ]:
df['country'].nunique()

In [ ]:
# Query to get the death rate and survival rate of lung cancer by country
query_death_rate = """
SELECT
    country,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    COUNT(*) as total_patients
FROM patients
GROUP BY country
ORDER BY non_survivors DESC;
"""

# Execute the query and load the data into a DataFrame
df_death_rate = pd.read_sql(query_death_rate, conn)

# Display the DataFrame
df_death_rate

In [ ]:
# Query to get the survival rate for patients with and without cirrhosis
query_cirrhosis_survival = """
SELECT
    cirrhosis,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    COUNT(*) as total_patients
FROM patients
GROUP BY cirrhosis
ORDER BY cirrhosis;
"""

# Execute the query and load the data into a DataFrame
df_cirrhosis_survival = pd.read_sql(query_cirrhosis_survival, conn)

# Display the DataFrame
df_cirrhosis_survival

#### Lung Cancer Survival Rates by Asthma Status

In [ ]:
# Query to get the survival rate for patients with and without asthma
query_asthma_survival = """
SELECT
    asthma,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    COUNT(*) as total_patients
FROM patients
GROUP BY asthma
ORDER BY asthma;
"""

# Execute the query and load the data into a DataFrame
df_asthma_survival = pd.read_sql(query_asthma_survival, conn)

# Display the DataFrame
df_asthma_survival

In [ ]:
# Query to get the survival rate for patients with and without other_cancer
query_other_cancer_survival = """
SELECT
    other_cancer,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    COUNT(*) as total_patients
FROM patients
GROUP BY other_cancer
ORDER BY other_cancer;
"""

# Execute the query and load the data into a DataFrame
df_other_cancer_survival = pd.read_sql(query_other_cancer_survival, conn)

# Display the DataFrame
df_other_cancer_survival

In [ ]:
# Query to get the survival rate for patients with and without hypertension
query_hypertension_survival = """
SELECT
    hypertension,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    COUNT(*) as total_patients
FROM patients
GROUP BY hypertension
ORDER BY hypertension;
"""

# Execute the query and load the data into a DataFrame
df_hypertension_survival = pd.read_sql(query_hypertension_survival, conn)

# Display the DataFrame
df_hypertension_survival

In [ ]:
# Query to get the survival rate for patients based on family history
query_family_history_survival = """
SELECT
    family_history,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    COUNT(*) as total_patients
FROM patients
GROUP BY family_history
ORDER BY family_history;
"""

# Execute the query and load the data into a DataFrame
df_family_history_survival = pd.read_sql(query_family_history_survival, conn)

# Display the DataFrame
df_family_history_survival

In [ ]:
# Query to get the survival rate for patients based on their gender
query_gender_survival = """
SELECT
    gender,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    COUNT(*) as total_patients
FROM patients
GROUP BY gender
ORDER BY gender;
"""

# Execute the query and load the data into a DataFrame
df_gender_survival = pd.read_sql(query_gender_survival, conn)

# Display the DataFrame
df_gender_survival

In [ ]:
# Query to get the survival rate for each smoking status category
query_smoking_survival = """
SELECT
    smoking_status,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    COUNT(*) as total_patients
FROM patients
GROUP BY smoking_status
ORDER BY non_survivors DESC;
"""

# Execute the query and load the data into a DataFrame
df_smoking_survival = pd.read_sql(query_smoking_survival, conn)

# Display the DataFrame
df_smoking_survival

In [ ]:
# Query to get the survival rate for each type of treatment
query_treatment_survival = """
SELECT
    treatment_type,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    COUNT(*) as total_patients
FROM patients
GROUP BY treatment_type
ORDER BY non_survivors DESC;
"""

# Execute the query and load the data into a DataFrame
df_treatment_survival = pd.read_sql(query_treatment_survival, conn)

# Display the DataFrame
df_treatment_survival

In [ ]:
# Query to get the survival rate for each cancer stage
query_cancer_stage_survival = """
SELECT
    cancer_stage,
    SUM(CASE WHEN survived = 'Yes' THEN 1 ELSE 0 END) as survivors,
    SUM(CASE WHEN survived = 'No' THEN 1 ELSE 0 END) as non_survivors,
    COUNT(*) as total_patients
FROM patients
GROUP BY cancer_stage
ORDER BY non_survivors DESC;
"""

# Execute the query and load the data into a DataFrame
df_cancer_stage_survival = pd.read_sql(query_cancer_stage_survival, conn)

# Display the DataFrame
df_cancer_stage_survival

In [ ]:
# Query to get BMI and cholesterol levels
query_bmi_cholesterol = """
SELECT
    bmi,
    cholesterol_level
FROM patients
LIMIT 10
"""

# Execute the query and load the data into a DataFrame
df_bmi_cholesterol = pd.read_sql(query_bmi_cholesterol, conn)

# Display the DataFrame
df_bmi_cholesterol

In [ ]:
query_bmi_cholesterol_survival = """
SELECT
    bmi,
    cholesterol_level,
    survived
FROM patients
LIMIT 10
"""

# Execute the query and load the data into a DataFrame
df_bmi_cholesterol_survival = pd.read_sql(query_bmi_cholesterol_survival, conn)

# Display the DataFrame
df_bmi_cholesterol_survival

In [ ]:
# Query to compare days_under_treatment for survivors based on treatment_type
query_survivors_treatment_days = """
SELECT 
    treatment_type,
    ROUND(AVG(days_under_treatment), 0) AS avg_days_under_treatment
FROM 
    patients
WHERE 
    survived = 'Yes'
GROUP BY 
    treatment_type
"""

# Execute the query and load the results into a DataFrame
df_survivors_treatment_days = pd.read_sql(query_survivors_treatment_days, conn)

# Display the DataFrame
df_survivors_treatment_days

In [ ]:
# Query to compare days_under_treatment for survivors based on treatment_type
query_survivors_cancer_stage = """
SELECT 
    cancer_stage,
    ROUND(AVG(days_under_treatment), 0) AS avg_days_under_treatment
FROM 
    patients
WHERE 
    survived = 'Yes'
GROUP BY 
    cancer_stage
"""

# Execute the query and load the results into a DataFrame
df_survivors_cancer_stage = pd.read_sql(query_survivors_cancer_stage, conn)

# Display the DataFrame
df_survivors_cancer_stage

In [ ]:
df.head()

In [ ]:
df.dtypes

In [43]:
# map all columns with 'Yes' and 'No' to 1 and 0 in order to be recognised by our algorithm
df['hypertension'] = df['hypertension'].map({'Yes': 1, 'No': 0})
df['asthma'] = df['asthma'].map({'Yes': 1, 'No': 0})
df['cirrhosis'] = df['cirrhosis'].map({'Yes': 1, 'No': 0})
df['other_cancer'] = df['other_cancer'].map({'Yes': 1, 'No': 0})
df['survived'] = df['survived'].map({'Yes': 1, 'No': 0})
df['family_history'] = df['family_history'].map({'Yes': 1, 'No': 0})

In [44]:
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})
df['cancer_stage'] = df['cancer_stage'].map({'Stage I': 1, 'Stage II': 2, 'Stage III':3, 'Stage IV':4})
df['treatment_type'] = df['treatment_type'].map({'Chemotherapy': 1, 'Combined': 2, 'Radiation':3, 'Surgery':4})
df['smoking_status'] = df['smoking_status'].map({'Current Smoker': 1, 'Former Smoker': 2, 'Never Smoked':3, 'Passive Smoker':4})

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
print(df['survived'].value_counts())

It has been observed that our target class is imbalanced. So, we’ll upsample the data so that the minority class matches the majority class.

In [48]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class 
df_majority = df[(df['survived']==0)] 
df_minority = df[(df['survived']==1)] 
# upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample with replacement
                                 n_samples= 2536298, # to match majority class
                                 random_state=42)  # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority])

After upsampling, the distribution of class is balanced as below

In [ ]:
print(df_upsampled['survived'].value_counts())

In [50]:
# Exclude the columns 'diagnosis_date', 'end_treatment_date', 'country', 'id', and 'survived'
X = df_upsampled[['age', 'gender', 'cancer_stage', 'family_history', 'smoking_status', 'bmi', 'cholesterol_level', 'hypertension', 'asthma', 'cirrhosis', 'other_cancer', 'treatment_type', 'days_under_treatment']]
y = df_upsampled['survived']

In [51]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

predicted_proba = rf_model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Initialize variables
step_factor = 0.05
threshold_values = np.arange(0.2, 0.81, step_factor)
roc_scores = []

# Iterate over each threshold and calculate the ROC AUC score
for threshold in threshold_values:
    predicted = (predicted_proba >= threshold).astype('int')
    roc_score = roc_auc_score(y_test, predicted)
    print(f'Threshold {threshold:.2f} -- ROC AUC: {roc_score:.4f}')
    roc_scores.append(roc_score)

# Find the best threshold
best_index = np.argmax(roc_scores)
best_threshold = threshold_values[best_index]
best_roc_score = roc_scores[best_index]

print(f'--- Optimum Threshold --- {best_threshold:.2f} -- ROC AUC: {best_roc_score:.4f}')

# Apply the best threshold to make final classifications
final_predictions = (predicted_proba >= best_threshold).astype('int')

# Evaluate the final classifications
final_accuracy = accuracy_score(y_test, final_predictions)
print(f'Final accuracy with optimal threshold {best_threshold:.2f}: {final_accuracy:.4f}')


In [55]:
import joblib
joblib.dump(rf_model,open('rfc_simp_model.joblib','wb'))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 200, 300,400,500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'criterion': ['gini' , 'entropy'],
    'max_samples':[None, 0.8, 0.9],
    'oob_score': [True, False]
}
rf = RandomForestClassifier()
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, 
                                   n_iter=50, cv=3, verbose=2, random_state=42, n_jobs= -1)
random_search.fit(X_train, y_train)
print("Best parameters:", random_search.best_params_)
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

In [58]:
import joblib
joblib.dump(best_rf,open('best_rf.joblib','wb'))

In [ ]:
from sklearn.metrics import confusion_matrix

# Get the confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)
print('Confusion Matrix:')
print(conf_matrix)
final_accuracy = accuracy_score(y_test, final_predictions)
print(f'Final accuracy with optimal threshold {best_threshold:.2f}: {final_accuracy:.4f}')

In [ ]:
import matplotlib.pyplot as plt

# Plot the confusion matrix
plt.matshow(conf_matrix, cmap=plt.cm.Blues)  # Use a colormap for better visualization
plt.colorbar()

# Add labels and title
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')

# Add ticks with class labels (assuming you have class labels stored somewhere)
plt.tick_params(bottom=False)  # Remove x-axis tick labels (predicted labels are shown on the top)
class_labels = ['Class 0', 'Class 1']  # Replace with your actual class labels
plt.xticks(range(len(class_labels)), class_labels, rotation=45)
plt.yticks(range(len(class_labels)), class_labels)

# Add values to each cell (optional)
for i in range(len(conf_matrix)):
  for j in range(len(conf_matrix[0])):
    plt.text(j, i, conf_matrix[i, j], ha='center', va='center')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

# Calculate mean squared error
mse = mean_squared_error(y_test, final_predictions)

# Print the MSE
print(f"Mean Squared Error: {mse:.2f}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

# Extract TP, TN, FP, FN
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
TP = conf_matrix[1, 1]

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, final_predictions)
recall = recall_score(y_test, final_predictions)
f1 = f1_score(y_test, final_predictions)

# Calculate specificity
specificity = TN / (TN + FP)

# Print metrics
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Specificity: {specificity:.4f}')

In [ ]:
from sklearn.metrics import roc_curve

# Calculate ROC curve points
fpr, tpr, thresholds = roc_curve(y_test, predicted_proba)

# Optional: Plot the ROC curve
plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % roc_auc_score(y_test, predicted_proba))
plt.title('ROC Curve')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
# Variable importance from random forest
importances = rf_model.feature_importances_
feat_importances = pd.Series(importances, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh', figsize=(10,6))
plt.title('Random Forest Variable Importance for Key Predictors')
plt.xlabel('Variable Importance (Gini)')
plt.ylabel('Predictor')
plt.show()

In [67]:
# Exclude the columns 'diagnosis_date', 'end_treatment_date', 'country', 'id', and 'survived'
X = df[['age', 'gender', 'cancer_stage', 'family_history', 'smoking_status', 'bmi', 'cholesterol_level', 'hypertension', 'asthma', 'cirrhosis', 'other_cancer', 'treatment_type', 'days_under_treatment']]
y = df['survived']

In [68]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Create an instance of BalancedBaggingClassifier
classifier = BalancedBaggingClassifier(
    estimator=DecisionTreeClassifier(),
    sampling_strategy='not majority',
    replacement=False,
    random_state=42
)

# Standardize the features if necessary
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the classifier to the training data
classifier.fit(X_train_scaled, y_train)

# Make predictions on the test data
preds = classifier.predict(X_test_scaled)

# Classification report
report = classification_report(y_test, preds)

# Confusion matrix
cm = confusion_matrix(y_test, preds)

# Accuracy score
accuracy = accuracy_score(y_test, preds)

# Print the results
print("Classification Report:\n", report)
print("Confusion Matrix:\n", cm)
print("Accuracy Score:", accuracy)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()